# Neural network general


Two perspectives on neural network representation. 

- Scalar-centered, in which single weights and units have their own nodes
- Vector-centered, which groups weights that we treat in the same way in the network.

For illustration purposes, neural networks are often depicted in their scalar form, but implemented using vectors. For large networks the vector representation has some advantages 

 

Feed-forward neural networks could be alternately named *sequential non-linear vector projections*.


## The purpose of non-linearities

## Forward pass

## Backward pass
